In [11]:
import os
os.path.dirname('.')
# os.path.realpath(__file__)

''

In [1]:
from torch.utils.data import Dataset, DataLoader
from nltk import sent_tokenize
import torch.nn as nn
import pandas as pd
import numpy as np
import itertools
import easydict
import MeCab
import torch
import json
import re

In [2]:
FILE_PATH = 'D:\\data\\text\\news-articles\\kbanker_articles_subtitles.csv'
CONFIG_PATH = 'config.json'
device = torch.device("cuda:0")
with open(CONFIG_PATH, 'r') as f:
    args = easydict.EasyDict(json.load(f))

In [3]:
#pre-processing shit
def pre_process_raw_article(article):
    """Pre-processing news articles.
    
    Args
        article (str): article text
    
    """
    replacements = [
        ('[“”]', '"'),
        ('[‘’]', '\''),
        ('\([^)]*\)', ''),
        ('[^가-힣\'"A-Za-z0-9.\s\?\!]', ' '),
        ('(?=[^0-9])\.(?=[^0-9])', '. '),
        ('\s\s+', ' ')
    ]
    
    for old, new in replacements:
        article = re.sub(old, new, article)
        
    return article

def mecab_tokenize(sentence):
    t = MeCab.Tagger()
    return [re.split(',', re.sub('\t', ',', s))[0] for s in t.parse(sentence).split('\n') if (s!='') & ('EOS' not in s)]

In [4]:
class NLPCorpusDataset(Dataset):
    """NLP Corpus dataset.
    
    Args:
        csv_file (str): Path to the csv file
        root_dir (str): root
        
    Attributes:
        root_dir (str): root
        word_to_idx (dict): word_to_idx mapping
        idx_to_word (dict): idx_to_word mapping
        x (list): train data (5-gram)
        y (list): label
        
    """

    def __init__(self, csv_file, root_dir):
        articles = pd.read_csv(csv_file, encoding='utf-8')['article'].dropna().values
        articles = [pre_process_raw_article(article) for article in articles]
        sentences = itertools.chain.from_iterable([sent_tokenize(article) for article in articles])
        corpus = [mecab_tokenize(s) for s in list(sentences)]
        self.root_dir = root_dir
        del articles
        del sentences
        
        #construct word matrix
        word_set = set(itertools.chain.from_iterable(corpus))
        self.word_to_idx = {word : idx for idx, word in enumerate(word_set)}
        self.idx_to_word = {self.word_to_idx[word] : word for word in self.word_to_idx}
        del word_set
        corpus = [[self.word_to_idx[word] for word in sentence] for sentence in corpus]
        
        #make train label dataset
        self.x = []
        self.y = []
        for sentence in corpus:
            for i in range(len(sentence) - args.window_size):
                self.x.append(sentence[i:i+args.window_size])
                self.y.append([sentence[i+args.window_size]])
        del corpus

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        return self.x[idx], self.y[idx]

In [ ]:
nlp_dataset = NLPCorpusDataset(csv_file=FILE_PATH, root_dir='.')

In [5]:
import pickle
# with open('kbanker_nlp_dataset.pkl', 'wb') as f:
#     pickle.dump(nlp_dataset, f)
    
with open('D:\\data\\text\\torch-dataset\\kbanker_nlp_dataset.pkl', 'rb') as f:
    nlp_dataset = pickle.load(f)

In [27]:
class EmbeddingModule(nn.Module):
    def __init__(self, vocab_size, embed_dim, h_dim):
        super(EmbeddingModule, self).__init__()
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.h_dim = h_dim
        
        self.embedding = nn.Embedding(vocab_size, embed_dim).float()
        self.embedding.weight.data.uniform_(-1,1)
        self.linear1 = nn.Linear(embed_dim*args.window_size, h_dim)
        self.linear1.weight.data.uniform_(-1,1)
        self.linear2 = nn.Linear(h_dim, vocab_size)
        self.linear2.weight.data.uniform_(-1,1)
        self.motorway = nn.Linear(embed_dim*args.window_size, vocab_size)
        self.motorway.weight.data.uniform_(-1,1)
        
        self.tanh = nn.Tanh()
    
    def forward(self, x):
        embedded = self.embedding(x).view(x.shape[0], -1)
        embedded.retain_grad()
        net = self.linear1(embedded)
        net = self.tanh(net)
        net = self.linear2(net)
        net = net + self.motorway(embedded)
        return net

In [28]:
def collate_fn(data):
    seqs, labels = zip(*data)
    return seqs, labels

dataloader = DataLoader(nlp_dataset, batch_size=args.batch_size, \
                        shuffle=False, num_workers=0, collate_fn=collate_fn)

model = EmbeddingModule(len(nlp_dataset.word_to_idx),\
                        args.embedding_dim, args.h_dim).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

for epoch in range(args.epoch):
    print('Epoch ' + str(epoch))
    for i, sample in enumerate(dataloader):
        x = torch.LongTensor(sample[0]).to(device)
        y = torch.LongTensor(sample[1]).view(-1).to(device)
        y_pred = model(x)
        
#         before = model.embedding.weight.data.cpu().numpy()
        
        loss = criterion(y_pred, y)
        
        optimizer.zero_grad()
        loss.backward(retain_graph=True)
        optimizer.step()
#         after = model.embedding.weight.data.cpu().numpy()
        
        if i % 100 == 99:
            print(i, loss.item())
#             print(model.embedding.weight.grad)
#             print((before-after).sum())
        
#         if i == 1000:
#             break

Epoch 0
99 62.67778015136719
199 63.74647903442383
299 63.39596176147461
399 64.41852569580078
499 61.38117599487305
599 64.03968811035156
699 63.008216857910156
799 62.92988967895508
899 63.75128173828125
999 64.544921875
1099 64.8699722290039
1199 63.41054916381836
1299 62.82123565673828
1399 63.57148361206055
1499 62.07074737548828
1599 62.1453742980957
1699 60.86100769042969
1799 63.75449752807617
1899 64.2336196899414
1999 62.802215576171875
2099 60.38370895385742
2199 62.24470138549805
2299 60.84575271606445
2399 65.15037536621094
2499 64.16695404052734
2599 61.73381042480469
2699 62.74930191040039
2799 59.87637710571289
2899 60.84674072265625
2999 64.46240997314453
3099 61.16950225830078
3199 65.25228881835938
3299 61.955177307128906
3399 62.765987396240234
3499 63.29732894897461
3599 65.3843765258789
3699 60.58243942260742
3799 63.23928451538086
3899 61.65533447265625
3999 62.545738220214844
4099 61.6417121887207
4199 60.36437225341797
4299 63.33218002319336
4399 62.66448974609

34399 54.31035232543945
34499 52.339683532714844
34599 48.95713424682617
34699 55.18548583984375
34799 55.697208404541016
34899 53.25103759765625
34999 53.03158950805664
35099 50.265663146972656
35199 54.6277961730957
35299 55.33652877807617
35399 54.21110534667969
35499 52.511627197265625
35599 51.987464904785156
35699 52.19077682495117
35799 50.55375671386719
35899 56.98297119140625
35999 56.53982162475586
36099 52.18104553222656
36199 56.43395233154297
36299 55.5566520690918
36399 48.08177947998047
36499 52.03126525878906
36599 54.03715515136719
36699 56.190223693847656
36799 55.39789581298828
36899 47.49184799194336
36999 57.57963943481445
37099 53.902225494384766
37199 57.000465393066406
37299 55.38442611694336
37399 56.018550872802734
37499 57.363441467285156
37599 56.985843658447266
37699 54.52055358886719
37799 47.25157165527344
37899 53.052391052246094
37999 49.40046310424805
38099 54.08591842651367
38199 53.43962860107422
38299 49.850433349609375
38399 55.649330139160156
3849

68299 52.628780364990234
68399 48.087528228759766
68499 50.02181625366211
68599 46.724082946777344
68699 45.9547233581543
68799 43.71285629272461
68899 50.93456268310547
68999 51.02065658569336
69099 49.10129928588867
69199 47.668338775634766
69299 47.26607131958008
69399 50.664710998535156
69499 46.09851837158203
69599 48.923301696777344
69699 52.69832229614258
69799 52.09811782836914
69899 51.37617874145508
69999 51.686302185058594
70099 47.00090789794922
70199 43.112754821777344
70299 48.006404876708984
70399 45.83015441894531
70499 50.10661697387695
70599 45.30277633666992
70699 50.84623336791992
70799 48.66859817504883
70899 47.417911529541016
70999 47.94688415527344
71099 47.59318161010742
71199 47.203250885009766
71299 41.26718521118164
71399 47.042423248291016
71499 49.365638732910156
71599 49.268043518066406
71699 49.71902084350586
71799 43.612693786621094
71899 41.57529830932617
71999 53.8401985168457
72099 47.501136779785156
72199 51.10138702392578
72299 45.59933853149414
72

102099 45.8990592956543
102199 44.250099182128906
102299 50.732940673828125
102399 37.363563537597656
102499 48.48688507080078
102599 44.97319412231445
102699 45.30730438232422
102799 48.00419998168945
102899 45.96885299682617
102999 46.93751525878906
103099 44.89738845825195
103199 43.420555114746094
103299 44.51301574707031
103399 45.90754699707031
103499 42.61838150024414
103599 43.841461181640625
103699 45.56271743774414
103799 44.361446380615234
103899 45.709903717041016
103999 40.39354705810547
104099 49.35417175292969
104199 41.582515716552734
104299 44.67423629760742
104399 42.20991134643555
104499 41.793758392333984
104599 46.924171447753906
104699 41.89620590209961
104799 46.20446014404297
104899 48.67446517944336
104999 47.4011344909668
105099 47.01268005371094
105199 45.64301300048828
105299 46.88358688354492
105399 50.493072509765625
105499 47.20439910888672
105599 44.36158752441406
105699 45.34138870239258
105799 45.391197204589844
105899 42.55223846435547
105999 43.76083

134699 46.84859848022461
134799 44.723941802978516
134899 41.52108383178711
134999 39.75914764404297
135099 42.65346145629883
135199 45.49260711669922
135299 41.44837188720703
135399 41.00776672363281
135499 39.74898910522461
135599 43.34931945800781
135699 38.381900787353516
135799 39.46189880371094
135899 39.323692321777344
135999 42.76273727416992
136099 38.43939971923828
136199 48.00178146362305
136299 37.84819030761719
136399 40.59420394897461
136499 41.27189636230469
136599 38.31846237182617
136699 40.00603103637695
136799 42.216766357421875
136899 36.92091751098633
136999 41.84341049194336
137099 39.70280075073242
137199 43.39857482910156
137299 44.81985092163086
137399 50.512298583984375
137499 39.590579986572266
137599 44.498443603515625
137699 40.2827033996582
137799 39.6891975402832
137899 40.33418273925781
137999 49.972267150878906
138099 45.35353088378906
138199 41.551719665527344
138299 41.017852783203125
138399 39.04316329956055
138499 43.59699249267578
138599 38.7690658

...

64399 7.291471481323242
64499 13.268452644348145
64599 11.846288681030273
64699 8.861592292785645
64799 11.38969612121582
64899 6.295368671417236
64999 6.668344974517822
65099 10.282760620117188
65199 8.371650695800781
65299 6.862942695617676
65399 7.73223352432251
65499 10.305059432983398
65599 7.917394638061523
65699 6.172475814819336
65799 9.138337135314941
65899 6.329061031341553
65999 6.695188045501709
66099 10.449030876159668
66199 9.920378684997559
66299 7.526142597198486
66399 13.994844436645508
66499 8.423798561096191
66599 6.843217849731445
66699 8.17004680633545
66799 9.2920560836792
66899 8.075039863586426
66999 8.46325969696045
67099 12.039544105529785
67199 13.774023056030273
67299 7.51912260055542
67399 9.147132873535156
67499 9.294992446899414
67599 9.828642845153809
67699 7.8937530517578125
67799 16.339677810668945
67899 8.02562427520752
67999 22.668819427490234
68099 10.00946044921875
68199 12.688482284545898
68299 9.67270278930664
68399 7.71332311630249
68499 8.51402

98499 7.284714698791504
98599 10.946495056152344
98699 11.20828628540039
98799 8.303502082824707
98899 6.245980739593506
98999 8.186368942260742
99099 11.14503288269043
99199 8.074005126953125
99299 10.101739883422852
99399 8.240840911865234
99499 8.06796646118164
99599 11.487316131591797
99699 6.633416652679443
99799 7.483216762542725
99899 7.597987174987793
99999 8.716911315917969
100099 9.028843879699707
100199 7.554734706878662
100299 11.21353816986084
100399 7.506083965301514
100499 8.470220565795898
100599 6.236276149749756
100699 7.402923583984375
100799 11.007662773132324
100899 6.669291019439697
100999 10.017489433288574
101099 9.791086196899414
101199 9.622332572937012
101299 9.017630577087402
101399 10.164507865905762
101499 11.863280296325684
101599 14.024751663208008
101699 9.900425910949707
101799 9.903403282165527
101899 8.291037559509277
101999 8.629264831542969
102099 7.498899936676025
102199 7.878593444824219
102299 9.951716423034668
102399 6.300424575805664
102499 10

131199 6.675961971282959
131299 10.083040237426758
131399 7.271777153015137
131499 8.640844345092773
131599 8.671147346496582
131699 7.82866096496582
131799 7.717639923095703
131899 7.0428547859191895
131999 9.22291374206543
132099 7.666478633880615
132199 7.140018939971924
132299 6.828649520874023
132399 8.935874938964844
132499 9.614501953125
132599 7.711052417755127
132699 7.8442063331604
132799 8.777921676635742
132899 11.117439270019531
132999 7.187025547027588
133099 8.276988983154297
133199 8.06503677368164
133299 7.6320366859436035
133399 8.447829246520996
133499 7.243034839630127
133599 9.834335327148438
133699 5.751479625701904
133799 8.665776252746582
133899 8.087493896484375
133999 6.573396682739258
134099 10.852349281311035
134199 8.70943832397461
134299 9.824736595153809
134399 11.122529983520508
134499 9.826786994934082
134599 11.494067192077637
134699 10.386748313903809
134799 9.807271957397461
134899 8.720773696899414
134999 6.998564720153809
135099 6.564076900482178
1

163899 9.61784839630127
163999 8.06578254699707
164099 14.09979248046875
164199 8.347413063049316
164299 6.746127605438232
164399 7.8304266929626465
164499 7.6346845626831055
164599 9.932287216186523
164699 13.665921211242676
164799 9.537976264953613
164899 8.77103328704834
164999 7.898249626159668
165099 9.36556625366211
165199 7.942676067352295
165299 6.543385982513428
165399 9.06258487701416
165499 10.692952156066895
165599 7.245687007904053
165699 13.195793151855469
165799 9.07093620300293
165899 8.070680618286133
165999 8.454877853393555
166099 7.699525833129883
166199 8.45667839050293
166299 8.84048843383789
166399 7.095271110534668
166499 8.449399948120117
166599 7.938308238983154
166699 11.448541641235352
166799 8.456573486328125
166899 8.818559646606445
166999 7.443279266357422
167099 9.932743072509766
167199 12.506123542785645
167299 9.209922790527344
167399 10.466775894165039
167499 7.93705415725708
167599 10.166220664978027
167699 10.538220405578613
167799 8.019350051879883

196699 8.61150074005127
196799 8.210057258605957
196899 8.21692943572998
196999 7.811959266662598
197099 11.674545288085938
197199 8.090049743652344
197299 11.419678688049316
197399 8.76314926147461
197499 6.761641979217529
197599 8.754899978637695
197699 7.089037895202637
197799 8.945480346679688
197899 9.100604057312012
197999 10.031332015991211
198099 9.768604278564453
198199 10.03651237487793
198299 6.268451690673828
198399 8.106674194335938
198499 9.761101722717285
198599 7.93717622756958
198699 10.961278915405273
198799 7.775957107543945
198899 10.371878623962402
198999 7.319029808044434
199099 9.95816707611084
199199 7.1088666915893555
199299 9.204633712768555
199399 7.634093284606934
199499 6.571288108825684
199599 10.392745971679688
199699 8.189997673034668
199799 6.182899475097656
199899 6.199772834777832
199999 7.089843273162842
200099 8.07576847076416
200199 13.63955307006836
200299 7.782471656799316
200399 8.510451316833496
200499 11.779239654541016
200599 7.90034866333007

In [32]:
import pickle

with open('nplm_model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [53]:
np.argmax(model(torch.LongTensor([nlp_dataset[1][0]]).to(device))[0].cpu().detach().numpy())

62995

In [54]:
nlp_dataset.idx_to_word[62995]

'할'

In [56]:
[nlp_dataset.idx_to_word[idx] for idx in nlp_dataset[0][0]]

['본점', '필수', '인력', '연수원', '에']

In [14]:
torch.LongTensor([[1],[2],[3],[4]]).view(1,-1)

tensor([[1, 2, 3, 4]])